In [9]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer

import pymongo


client = pymongo.MongoClient('mongodb://donkey:27017/')
db = client.sphere_kaggle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
db.collection_names()

[u'debug',
 u'errors',
 u'results',
 u'system.indexes',
 u'test',
 u'test_hashtag_data',
 u'test_text_data',
 u'test_tweets',
 u'train_hashtag_data',
 u'train_text_data',
 u'train_tweets',
 u'train_tweets_tmp',
 u'tweets',
 u'unrecognized_locations',
 u'users',
 u'vocabulary',
 u'words_log_probs']

In [10]:
label_1_counter = Counter(db.results.find_one({'_id':'label_1_counter'})["data"])
label_2_counter = Counter(db.results.find_one({'_id':'label_2_counter'})["data"])
label_3_counter = Counter(db.results.find_one({'_id':'label_3_counter'})["data"])

In [ ]:
uniq_1 = label_1_counter - (label_2_counter + label_3_counter)
uniq_2 = label_2_counter - (label_1_counter + label_3_counter)
uniq_3 = label_3_counter - (label_1_counter + label_2_counter)

In [17]:
def get_dataframe(class_log_prob, name):
    d = pd.DataFrame(class_log_prob)
    d.index = d[0]
    d.index.name = "words"
    d.drop(0, axis=1, inplace=True)
    d.columns = [name]
    
    return d

In [18]:
def reduce_uniq_size(uniq, counter=20):
    words, count = zip(*uniq.most_common())
    count = np.array(count)
    words = np.array(words)
    
    indicies = count > counter
    
    return zip(words[indicies], count[indicies])
    

In [ ]:
fd1 = get_dataframe(reduce_uniq_size(uniq_1), "class_log_prob_1")
fd2 = get_dataframe(reduce_uniq_size(uniq_2), "class_log_prob_2")
fd3 = get_dataframe(reduce_uniq_size(uniq_3), "class_log_prob_3")


In [11]:
def get_log_prob_from_neg_counter(counter):
    counts = np.array(zip(*counter.most_common())[1])
    counts += np.abs(np.min(counts)) + 1
    return zip(zip(*counter.most_common())[0], np.log(counts) - np.log(np.sum(counts)))

In [12]:
first_uniq = label_1_counter.copy()
first_uniq.subtract(label_2_counter + label_3_counter)

In [13]:
second_uniq = label_2_counter.copy()
second_uniq.subtract(label_1_counter + label_3_counter)

In [14]:
third_uniq = label_3_counter.copy()
third_uniq.subtract(label_1_counter + label_2_counter)

In [15]:
class_1_log_prob = get_log_prob_from_neg_counter(first_uniq)
class_2_log_prob = get_log_prob_from_neg_counter(second_uniq)
class_3_log_prob = get_log_prob_from_neg_counter(third_uniq)

In [19]:
df1 = get_dataframe(class_1_log_prob, "class_1_log_prob")
df2 = get_dataframe(class_2_log_prob, "class_2_log_prob")
df3 = get_dataframe(class_3_log_prob, "class_3_log_prob")

In [20]:
words_df = pd.concat([df1, df2, df3], axis=1, join='inner')

In [21]:
words_df.head()

,class_1_log_prob,class_2_log_prob,class_3_log_prob
words,,,
sheldon,-21.373426,-12.963560,-12.278134
bang,-13.329121,-12.879284,-12.562455
theori,-13.280881,-12.860424,-12.573130
follow,-14.542552,-13.399957,-12.579019
#bigbangtheori,-13.184737,-12.843587,-12.593587


In [22]:
from sklearn.feature_extraction.text import CountVectorizer

In [24]:
cv = CountVectorizer(vocabulary=words_df.index.tolist())

In [33]:
user_data = db.train_text_data.find()[2]
user_data.get("label")

3

In [34]:
d = cv.transform([user_data["text"]]).dot(words_df.as_matrix())

In [36]:
np.argmax(d)

2